In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
!pip install tensorflow-gpu


     |████████████████████████████████| 516.2MB 31kB/s 


In [3]:
from keras.datasets import cifar10

Using TensorFlow backend.


In [4]:
(X_train, y_train) , (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 6s 0us/step


In [5]:
print('X_train shape - ',X_train.shape)
print('X_test shape - ',X_test.shape)
print('y_train shape - ',y_train.shape)
print('y_test shape - ',y_test.shape)

X_train shape -  (50000, 32, 32, 3)
X_test shape -  (10000, 32, 32, 3)
y_train shape -  (50000, 1)
y_test shape -  (10000, 1)


In [6]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train/255
X_test = X_test/255

In [7]:
import keras
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [8]:
Input_shape = X_train.shape[1:]
Input_shape

(32, 32, 3)

In [9]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D,Dense,AveragePooling2D,MaxPooling2D, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
from keras.layers import BatchNormalization

In [10]:
cnn_model = Sequential()
cnn_model.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', kernel_initializer='he_uniform', padding='same',input_shape = Input_shape))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', kernel_initializer='he_uniform',padding='same'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D(2,2))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', kernel_initializer='he_uniform', padding='same',input_shape = Input_shape))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', kernel_initializer='he_uniform',padding='same'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D(2,2))
cnn_model.add(Dropout(0.3))


cnn_model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu', kernel_initializer='he_uniform',padding='same'))
cnn_model.add(BatchNormalization())
cnn_model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu', kernel_initializer='he_uniform',padding='same'))
cnn_model.add(BatchNormalization())
cnn_model.add(AveragePooling2D(2,2))
cnn_model.add(Dropout(0.4))

cnn_model.add(Flatten())

cnn_model.add(Dense(units = 1024, activation = 'relu', kernel_initializer='he_uniform'))
cnn_model.add(BatchNormalization())
cnn_model.add(Dropout(0.5))

cnn_model.add(Dense(units = 10, activation = 'softmax'))

In [11]:
cnn_model.compile(loss = 'categorical_crossentropy', optimizer = Adam(lr=0.001,beta_1=0.9,beta_2=0.999), metrics = ['accuracy'])

In [12]:
cnn_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)       

In [13]:
history = cnn_model.fit(X_train, y_train, batch_size = 64, epochs = 20, shuffle = True,validation_data=(X_test,y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 49s 972us/step - loss: 1.6714 - accuracy: 0.4649 - val_loss: 1.2315 - val_accuracy: 0.5827
Epoch 2/20
50000/50000 [==============================] - 41s 818us/step - loss: 1.0697 - accuracy: 0.6270 - val_loss: 0.8894 - val_accuracy: 0.6835
Epoch 3/20
50000/50000 [==============================] - 41s 819us/step - loss: 0.8621 - accuracy: 0.6971 - val_loss: 0.7227 - val_accuracy: 0.7540
Epoch 4/20
50000/50000 [==============================] - 41s 812us/step - loss: 0.7529 - accuracy: 0.7364 - val_loss: 0.6943 - val_accuracy: 0.7632
Epoch 5/20
50000/50000 [==============================] - 41s 818us/step - loss: 0.6743 - accuracy: 0.7652 - val_loss: 0.6109 - val_accuracy: 0.7922
Epoch 6/20
50000/50000 [==============================] - 41s 819us/step - loss: 0.6228 - accuracy: 0.7834 - val_loss: 0.5986 - val_accuracy: 0.7928
Epoch 7/20
50000/50000 [==============================] 

In [14]:
evaluation = cnn_model.evaluate(X_test, y_test)
print('Test Accuracy: {}'.format(evaluation[1]))

10000/10000 [==============================] - 3s 331us/step
Test Accuracy: 0.8546000123023987


In [15]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
                            
                            
                            width_shift_range=0.1,
                             height_shift_range=0.1,
                             shear_range=0.1,
                             zoom_range=0.1,
                            horizontal_flip = True,
                            
                             )
datagen.fit(X_train)

In [16]:
cnn_model.fit_generator(datagen.flow(X_train, y_train, batch_size = 64), epochs = 200)

Epoch 1/200
782/782 [==============================] - 50s 64ms/step - loss: 0.6120 - accuracy: 0.7955
Epoch 2/200
782/782 [==============================] - 50s 64ms/step - loss: 0.5443 - accuracy: 0.8155
Epoch 3/200
782/782 [==============================] - 50s 64ms/step - loss: 0.5287 - accuracy: 0.8178
Epoch 4/200
782/782 [==============================] - 50s 64ms/step - loss: 0.5144 - accuracy: 0.8234
Epoch 5/200
782/782 [==============================] - 50s 65ms/step - loss: 0.5068 - accuracy: 0.8234
Epoch 6/200
782/782 [==============================] - 50s 64ms/step - loss: 0.4871 - accuracy: 0.8323
Epoch 7/200
782/782 [==============================] - 50s 64ms/step - loss: 0.4837 - accuracy: 0.8316
Epoch 8/200
782/782 [==============================] - 49s 63ms/step - loss: 0.4746 - accuracy: 0.8374
Epoch 9/200
782/782 [==============================] - 49s 63ms/step - loss: 0.4611 - accuracy: 0.8389
Epoch 10/200
782/782 [==============================] - 50s 63ms/step - l

In [17]:
score = cnn_model.evaluate(X_test, y_test)

10000/10000 [==============================] - 3s 338us/step


In [19]:
print('Test accuracy', score[1])

Test accuracy 0.8942000269889832


In [20]:
score_test= cnn_model.evaluate(X_train, y_train)

50000/50000 [==============================] - 19s 380us/step


In [21]:
print('Train accuracy', score_test[1])

Train accuracy 0.9718000292778015
